# Pt 3: Create a Travel Itinerary with a Corresponding Map

In [1]:
# Import dependencies
import pandas as pd
import gmaps
import gmaps.datasets
import requests

from config import g_key

# Configure gmaps to use my Google API Key
gmaps.configure(api_key=g_key)

In [2]:
# Open File
vacation_df = pd.read_csv("challenge_data/WeatherPy_vacation.csv")
vacation_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,1,Sao Filipe,CV,68.52,overcast clouds,14.90,-24.50,Tortuga B&B
1,3,Bahir Dar,ET,75.31,scattered clouds,11.59,37.39,"Delano Hotel, Bahir Dar"
2,7,Matara,LK,75.51,overcast clouds,5.95,80.54,Amaloh Boutique Resort
3,9,Hilo,US,70.00,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
4,10,Vaini,TO,77.00,broken clouds,-21.20,-175.20,Keleti Beach Resort
5,17,Saldanha,ZA,78.80,clear sky,-33.01,17.94,Blue Bay Lodge
6,25,Kapaa,US,69.80,clear sky,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
7,33,Port Elizabeth,ZA,62.60,clear sky,-33.92,25.57,39 On Nile Guest House
8,35,Saint-Philippe,RE,77.00,clear sky,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
9,36,Rio Gallegos,AR,62.60,scattered clouds,-51.62,-69.22,Hotel Santa Cruz


In [3]:
# Filter the DataFrame for each city you want to go to and create separate DataFrames for each city.
sechura_city_df = vacation_df.loc[vacation_df["City"] == "Sechura"]
pimentel_city_df = vacation_df.loc[vacation_df["City"] == "Pimentel"]
chicama_city_df = vacation_df.loc[vacation_df["City"] == "Chicama"]
pisco_city_df = vacation_df.loc[vacation_df["City"] == "Pisco"]

In [4]:
# Get the latitude-longitude pairs for each city DataFrame as tuples
sechura = tuple(sechura_city_df[["Lat", "Lng"]].values[0])
pimentel = tuple(pimentel_city_df[["Lat", "Lng"]].values[0])
chicama = tuple(chicama_city_df[["Lat", "Lng"]].values[0])
pisco = tuple(pisco_city_df[["Lat", "Lng"]].values[0])

In [5]:
# Create a directions layer map; draw routes based on the Google maps directions service
fig = gmaps.figure()
sechura2pisco = gmaps.directions_layer(
        sechura, pisco, waypoints=[pimentel, chicama],
        travel_mode='DRIVING')
fig.add_layer(sechura2pisco)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Create a new DataFrame that has all the individual city DataFrames you created
peru_trip_df = pd.DataFrame()
peru_trip_df = peru_trip_df.append([sechura_city_df, pimentel_city_df, chicama_city_df, pisco_city_df])
peru_trip_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
92,334,Sechura,PE,75.51,scattered clouds,-5.56,-80.82,Hospedaje Costa Bella
122,445,Pimentel,PE,66.20,shower rain,-6.84,-79.93,Casona Del Muelle
109,394,Chicama,PE,73.40,haze,-7.84,-79.15,Hospedaje Punta Cana PERU
78,271,Pisco,PE,78.01,scattered clouds,-13.70,-76.22,Hostal San Isidro Oficial


In [17]:
# Create a marker layer map for the four cities

# 1. Description Definition
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 2. Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in peru_trip_df.iterrows()]

# 3. Add marker layer map
locations = peru_trip_df[["Lat", "Lng"]]

fig = gmaps.figure()

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

sechura2pisco = gmaps.directions_layer(sechura, pisco, 
                                       waypoints=[pimentel, chicama],
                                       travel_mode='DRIVING')

fig.add_layer(marker_layer)
#fig.add_layer(sechura2pisco)

# 4. Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))